In [1]:
import numpy as np
import pandas as pd
import pywt

In [ ]:
csv_file = 'xa.s12.00.mhz.1970-06-26HR00_evid00009.csv'  
df = pd.read_csv(csv_file)

rel_time = df['time_rel(sec)']  # relative times
velocity = df['velocity(m/s)']  # velocity

# sampling rate
fs = 1 / rel_time.diff().mean() 

# Wavelet Denoising Function
def wavelet_denoise(signal, wavelet='db8', level=2):
    coeff = pywt.wavedec(signal, wavelet, mode='per', level=level)
    # Resetting the details of the transformation coefficients (noise)
    coeff[1:] = [np.zeros_like(c) for c in coeff[1:]]
    reconstructed_signal = pywt.waverec(coeff, wavelet, mode='per')
    
    # Equalize length with original signal
    if len(reconstructed_signal) > len(signal):
        reconstructed_signal = reconstructed_signal[:len(signal)]
    elif len(reconstructed_signal) < len(signal):
        reconstructed_signal = np.pad(reconstructed_signal, (0, len(signal) - len(reconstructed_signal)), 'constant')
    
    return reconstructed_signal

# power spectrum calculation
def compute_power_spectrum(signal, fs):
    # FFT'yi hesapla
    fft_values = np.fft.fft(signal)
    power_spectrum = np.abs(fft_values) ** 2
    freq = np.fft.fftfreq(len(signal), 1/fs)
    return freq[:len(freq)//2], power_spectrum[:len(power_spectrum)//2]  # Pozitif frekanslar

# apply filter
denoised_velocity = wavelet_denoise(velocity, wavelet='db8', level=2)

freqs, power_spectrum = compute_power_spectrum(denoised_velocity, fs)

# adding spectrum values to dataframe
df['fft_freqs(Hz)'] = pd.Series(freqs[:len(df)])  # Frekanslar
df['power_spectrum'] = pd.Series(power_spectrum[:len(df)])  # Güç spektrumu değerleri

# saving as a csv file
output_csv = 'p.csv'
df.to_csv(output_csv, index=False)